In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD
import cv2
import numpy as np


In [7]:
train_datagen_with_aug = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=[0.2,0.5],
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   validation_split=0.2)

train_generator = train_datagen_with_aug.flow_from_directory(
    r'/content/drive/MyDrive/DR-IMG_SET',
    target_size=(224, 224),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='categorical',
    subset='training'
    ) # set as training data

validation_generator = train_datagen_with_aug.flow_from_directory(
    r'/content/drive/MyDrive/DR-IMG_SET', # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    # color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
    ) # set as validation data

input_img = Input(shape=(224, 224, 3))

Found 2248 images belonging to 5 classes.
Found 562 images belonging to 5 classes.


In [8]:
from keras.applications import densenet

DENSENET = Sequential()

densenet_model = keras.applications.densenet.DenseNet121(
    include_top=False, weights='imagenet',
    input_shape=(224,224,3), pooling='max', classes=5
)

29084464/29084464 [==============================] - 1s 0us/step


In [9]:
for layer in densenet_model.layers:
    layer.trainable = False

DENSENET.add(densenet_model)
DENSENET.add(Flatten())
DENSENET.add(Dense(512,activation='relu'))
DENSENET.add(Dense(5, activation='softmax'))

DENSENET.summary()

#adam = keras.optimizers.Adam(learning_rate=0.01)
SGD = keras.optimizers.SGD(learning_rate=0.0001)
DENSENET.compile(optimizer='SGD',loss='categorical_crossentropy', metrics=['accuracy'])

graph = DENSENET.fit(train_generator, epochs=25, validation_data = validation_generator)
graph



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 7564869 (28.86 MB)
Trainable params: 527365 (2.01 MB)
Non-trainable params: 7037504 (26.85 MB)
_________________________________________________________________
Epoch 1/25
71/71 [==============================] - 979s 14s/step - loss: 1.7392 - accuracy: 0.4760 - val_loss: 1.3020 - val_accuracy: 0.5658
Epo

In [1]:

import matplotlib.pyplot as plt

fig1 = plt.gcf()

plt.plot(graph.history['accuracy'])
plt.plot(graph.history['val_accuracy'])

# plt.axis(ymin=0.4, ymax=1)
plt.grid()

plt.title('DenseNet121 Model Accuracy for AGRI Dataset')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'])

plt.show()

NameError: ignored

<Figure size 640x480 with 0 Axes>

In [2]:
max_acc = max(graph.history['val_accuracy'])

print('The highest accuracy achieved using DenseNet121 model is',max_acc*100)

NameError: ignored